In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import time

import ormsgpack
from sqlalchemy import create_engine, select
from sqlalchemy.orm import Session

from models import File, StorageDevice, Task

engine = create_engine("sqlite:///database.db")
import zmq

from tasks import start_task
from time import sleep
from contextlib import contextmanager

In [3]:
with Session(engine) as session:
    sd = StorageDevice(
        name="T7",
        base_path="/Volumes/T7/",
        sync_on_deck=True,
        sync_continue_watching=True,
        sync_playlist="Download to SSD",
    )
    session.add(sd)
    session.commit()

In [6]:
context = zmq.Context()
#  Socket to talk to server
print("Connecting to hello world server…")
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:5555")

with Session(engine) as session:
    sd = session.query(StorageDevice).filter_by(name="T7").first()
    files = session.query(File).filter_by(storage_device_id=sd.id).all()
    for file in files:
        file.storage_path = sd.get_drive_path(file.remote_path)
        session.add(file)
        session.commit()
        if not os.path.exists(file.local_path):
            raise Exception("File does not exist: {}".format(file.storage_path))
        if not os.path.exists(file.storage_path) or os.path.getsize(
            file.storage_path
        ) != os.path.getsize(file.local_path):
            task = Task(
                name="Transfer {} to {}".format(file.title, sd.name),
                func="transfer_file",
                args=[file.local_path, file.storage_path],
                kwargs={},
            )
            session.add(task)
            session.commit()
            #  Send reply back to client
            start_task(task.id)

Connecting to hello world server…


In [9]:
with Session(engine) as session:
    tasks = session.query(Task).all()
    for task in tasks:
        if task.progress > 0 and task.progress < task.total:
            print(
                "{:.2%}\t'{}'".format(
                    task.progress / task.total, json.loads(task.args)[0]
                )
            )

In [12]:
with Session(engine) as session:
    # task = Task(
    #     name="Check Files",
    #     func="check_files",
    #     args=[1],
    #     kwargs={},
    # )
    # session.add(task)
    # session.commit()
    # start_task(task.id)
    task = Task(
        name="Transfer Files",
        func="transfer_files",
        args=[1],
        kwargs={},
    )
    session.add(task)
    session.commit()
    start_task(task.id)

82


In [10]:
from sqlalchemy import select

with Session(engine) as session:
    task = session.execute(select(Task).where(Task.id == 1))
    print(task.scalar_one_or_none())

In [15]:
with Session(engine) as session:
    stmnt = select(StorageDevice).where(StorageDevice.id == 1)
    print(stmnt)
    sd = session.execute(
        select(StorageDevice).where(StorageDevice.id == 1)
    ).scalar_one_or_none()
    print(sd)

SELECT storage_device.id, storage_device.name, storage_device.base_path, storage_device.sync_on_deck, storage_device.sync_continue_watching, storage_device.sync_playlist 
FROM storage_device 
WHERE storage_device.id = :id_1
None


In [24]:
with Session(engine) as session:
    files = (
        session.execute(
            select(File).where(File.storage_device_id == 1, File.storage_path != None)
        )
        .scalars()
        .all()
    )

[]


In [23]:
keep = 0
remove = 0
with Session(engine) as session:
    for path, dirs, files in list(os.walk("/Volumes/T7/plex files/"))[::-1]:
        if len(files):
            for file in files:
                p = os.path.join(path, file)
                f = session.execute(
                    select(File).where(File.storage_path == p)
                ).scalar_one_or_none()
                if f is None:
                    print("Removing '{}' as it's not in the database".format(p))
                    os.remove(p)
                    remove += 1
                else:
                    keep += 1
print("Keep: {}".format(keep))
print("Remove: {}".format(remove))
# os.remove(p)

Keep: 179
Remove: 0


In [9]:
with Session(engine) as session:
    sd = session.query(StorageDevice).filter_by(id=1).first()
    files = session.query(File).filter_by(storage_device_id=sd.id).all()


In [7]:
    for file in files:
        file.storage_path = sd.get_drive_path(file.remote_path)
        session.add(file)
        session.commit()
        if not os.path.exists(file.local_path):
            raise Exception("File does not exist: {}".format(file.storage_path))
        if not os.path.exists(file.storage_path) or os.path.getsize(
            file.storage_path
        ) != os.path.getsize(file.local_path):
            task = Task(
                name="Transfer {} to {}".format(file.title, sd.name),
                func="transfer_file",
                args=[file.local_path, file.storage_path],
                kwargs={},
            )
            session.add(task)
            session.commit()
            #  Send reply back to client
            start_task(task.id)

82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82
82


In [8]:
with Session(engine) as session:
    sd = session.query(StorageDevice).filter_by(id=1).first()
    files = session.query(File).filter_by(storage_device_id=sd.id).all()
    for file in files:
        file.storage_path = file.storage_path.replace("/Volumes/T7","/Volumes/T7/")
    session.commit()


In [5]:
with Session(engine) as session:
    files = session.query(File).all()
    for file in files:
        file.rating_key = int(file.rating_key)
        session.commit()